# Analysis Project on Data : Global Conflict Hashtag on Social Media
- **Linkedin :** Muhammad Aditya Bayhaqie
- **Email :** adityabayhaqie@gmail.com
- **Github :** bayhaqieee

## Business Questions

- When is the Highest Posting time and Lowest Posting time for the Most Hashtag?
- Which Conflict shows a High Exposure on Hashtag?
- What Content Summary for each Tag?